In [8]:
import os
import numpy as np
import zipfile
from PIL import Image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model
from sklearn.ensemble import RandomForestClassifier
import joblib  # Used for saving the model
from google.colab import files

# Upload and unzip the ZIP file
uploaded = files.upload()
for zip_file in uploaded.keys():
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall('images')

# Initialize ResNet50 model for feature extraction
base_model = ResNet50(weights='imagenet', include_top=False)
model = Model(inputs=base_model.input, outputs=base_model.output)

# Function: Preprocess image
def process_image(image_path):
    image = Image.open(image_path)
    if image.mode != 'RGB':
        image = image.convert('RGB')
    image_resized = image.resize((224, 224))
    image_array = np.array(image_resized)
    image_preprocessed = preprocess_input(image_array)
    return image_preprocessed

# Function: Extract features from images
def extract_features(data_dir):
    features = []
    image_paths = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.lower().endswith('.tif')]
    for img_path in image_paths:
        processed_img = process_image(img_path)
        features.append(model.predict(np.expand_dims(processed_img, axis=0)).flatten())
    return np.array(features)

# Extract features
features = extract_features('images')

# All image labels are "Bahrfeldt"
labels = ["Bahrfeldt"] * len(features)  # Create a list with the same length as features, all elements are "Bahrfeldt"

# Train the classifier using features and labels
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(features, labels)

# Save the trained classifier model, file name includes the label name
classifier_model_filename = 'classifier_Bahrfeldt.pkl'
joblib.dump(classifier, classifier_model_filename)
print(f"Classifier model saved as: {classifier_model_filename}")

# Download the classifier model to your local system
files.download(classifier_model_filename)


Saving Bahrfeldt-19-a.zip to Bahrfeldt-19-a (6).zip


1/1 [==============================] - 0s 128ms/step
Classifier model saved as: classifier_Bahrfeldt.pkl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>